In [1]:
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u

from galpy.orbit import Orbit
from astroML.density_estimation import XDGMM

## Load Data

In [2]:
data_ic = Table.read("data/ic348_apogee.csv")
data_ngc = Table.read("data/ngc1333_apogee.csv")

In [3]:
# sun orbit data -- used later to convert between uvw and UVW
osun = Orbit([0,0,0,-11.1, -12.24, -7.25],radec=True,uvw=True)

## Functions

##### calculate LBD values for each star

In [4]:
def calculate_LBDUVW(row):
    samp = np.array([row['ra'], row['dec'], row['parallax'], row['pmra'], row['pmdec'], row['RV']])
    
    coord = SkyCoord(ra = samp[0] * u.deg, dec = samp[1] * u.deg, distance=1000/samp[2] *u.pc,
                             pm_ra_cosdec=samp[3] *u.mas/u.yr, pm_dec=samp[4] *u.mas/u.yr,
                             radial_velocity=samp[5] *u.km/u.s,
                             frame='icrs')
    
    o = Orbit(coord)
    
    return coord.galactic.l.value, coord.galactic.b.value, coord.galactic.distance.value, o.U(), o.V(), o.W()

##### calculate LBD and UVW errors for each star

In [5]:
def calculate_err(row):
    mean = np.array([row['ra'], row['dec'], row['parallax'], row['pmra'], row['pmdec'], row['RV']])
    std = np.array([0, 0, row['parallax_error'], row['pmra_error'], row['pmdec_error'], row['e_RV']])
    
    X = np.vstack(mean)
    Xerr = np.zeros((6,6))
    np.fill_diagonal(Xerr, std)
    
    single_star_samps = np.random.multivariate_normal(mean, Xerr**2, size=100, check_valid='warn', tol=1e-8)
    
    coords = []
    for samp in single_star_samps:
        if samp[2] < 0:
            pass
        else:
            coords.append(SkyCoord(ra = samp[0] * u.deg, dec = samp[1] * u.deg, distance=1000/samp[2] *u.pc,
                             pm_ra_cosdec=samp[3] *u.mas/u.yr, pm_dec=samp[4] *u.mas/u.yr,
                             radial_velocity=samp[5] *u.km/u.s,
                             frame='icrs'))
    
    l_list = []
    d_list = []
    b_list = []
    U_list = []
    V_list = []
    W_list = []

    for coord in coords:
        l_list.append(coord.galactic.l.value)
        b_list.append(coord.galactic.b.value)
        d_list.append(coord.galactic.distance.value)
        
        o = Orbit(coord)
        U_list.append(o.U())
        V_list.append(o.V())
        W_list.append(o.W())
        
    l_err = np.std(l_list)
    b_err = np.std(b_list)
    d_err = np.std(d_list)
    
    U_err = np.std(U_list)
    V_err = np.std(V_list)
    W_err = np.std(W_list)

    
    return l_err, b_err, d_err, U_err, V_err, W_err

## Calculate Coords and Errors for Clusters

In [6]:
l_err_ic = []
b_err_ic = []
d_err_ic = []

U_err_ic = []
V_err_ic = []
W_err_ic = []

L_ic = []
B_ic = []
D_ic = []

U_ic = []
V_ic = []
W_ic = []

count = 1

for row in data_ic:
    print(count)
    l_err, b_err, d_err, U_err, V_err, W_err = calculate_err(row)
    L,B,D, U, V, W= calculate_LBDUVW(row)
    
    l_err_ic.append(l_err)
    b_err_ic.append(b_err)
    d_err_ic.append(d_err)
    
    U_err_ic.append(U_err)
    V_err_ic.append(V_err)
    W_err_ic.append(W_err)
    
    L_ic.append(L)
    B_ic.append(B)
    D_ic.append(D)
    
    U_ic.append(U)
    V_ic.append(V)
    W_ic.append(W)
    
    count += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151


In [7]:
l_err_ngc = []
b_err_ngc = []
d_err_ngc = []

U_err_ngc = []
V_err_ngc = []
W_err_ngc = []

L_ngc = []
B_ngc = []
D_ngc = []

U_ngc = []
V_ngc = []
W_ngc = []

count = 1

for row in data_ngc:
    print(count)
    l_err, b_err, d_err, U_err, V_err, W_err = calculate_err(row)
    L,B,D,U,V,W = calculate_LBDUVW(row)
    
    l_err_ngc.append(l_err)
    b_err_ngc.append(b_err)
    d_err_ngc.append(d_err)
    
    U_err_ngc.append(U_err)
    V_err_ngc.append(V_err)
    W_err_ngc.append(W_err)
    
    L_ngc.append(L)
    B_ngc.append(B)
    D_ngc.append(D)
    
    U_ngc.append(U)
    V_ngc.append(V)
    W_ngc.append(W)
    
    count += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57


## Running Deconvolution

In [8]:
X_ic = np.vstack([L_ic, B_ic, D_ic, U_ic, V_ic, W_ic]).T
Xerr_ic = np.zeros(X_ic.shape + X_ic.shape[-1:])

for i in range(0,len(l_err_ic)):
    np.fill_diagonal(Xerr_ic[i], [l_err_ic[i]**2, b_err_ic[i]**2, d_err_ic[i]**2,
                              U_err_ic[i]**2, V_err_ic[i]**2, W_err_ic[i]**2])

clf_ic = XDGMM(n_components = 1)
clf_ic.fit(X_ic, Xerr_ic)

mu_ic = clf_ic.mu[0]
cov_ic = np.diagonal(clf_ic.V[0])
sigma_ic = np.sqrt(np.diagonal(clf_ic.V[0]))

In [9]:
X_ngc = np.vstack([L_ngc, B_ngc, D_ngc, U_ngc, V_ngc, W_ngc]).T
Xerr_ngc = np.zeros(X_ngc.shape + X_ngc.shape[-1:])

for i in range(0,len(l_err_ngc)):
    np.fill_diagonal(Xerr_ngc[i], [l_err_ngc[i]**2, b_err_ngc[i]**2, d_err_ngc[i]**2,
                              U_err_ngc[i]**2, V_err_ngc[i]**2, W_err_ngc[i]**2])

clf_ngc = XDGMM(n_components = 1)
clf_ngc.fit(X_ngc, Xerr_ngc)

mu_ngc = clf_ngc.mu[0]
cov_ngc = np.diagonal(clf_ngc.V[0])
sigma_ngc = np.sqrt(np.diagonal(clf_ngc.V[0]))

## Deconvolution Results

In [10]:
print("IC 348 summary")
print(f"means   L:{mu_ic[0]} --  B:{mu_ic[1]} -- D:{mu_ic[2]}")
print(f"        U:{mu_ic[3]} --  V:{mu_ic[4]} --  W:{mu_ic[5]}")
print(f"        u:{mu_ic[3] - osun.U()} -- v:{mu_ic[4] - osun.V()} -- w:{mu_ic[5] - osun.W()}")
print(f"sigma   L:{sigma_ic[0]} --  B:{sigma_ic[1]} -- D:{sigma_ic[2]}")
print(f"        U:{sigma_ic[3]} --  V:{sigma_ic[4]} --  W:{sigma_ic[5]}")

print("")

print("NGC 1333 summary")
print(f"means   L:{mu_ngc[0]} --  B:{mu_ngc[1]} -- D:{mu_ngc[2]}")
print(f"        U:{mu_ngc[3]} --  V:{mu_ngc[4]} --  W:{mu_ngc[5]}")
print(f"        u:{mu_ngc[3] - osun.U()} -- v:{mu_ngc[4] - osun.V()} -- w:{mu_ngc[5] - osun.W()}")
print(f"sigma   L:{sigma_ngc[0]} --  B:{sigma_ngc[1]} -- D:{sigma_ngc[2]}")
print(f"        U:{sigma_ngc[3]} --  V:{sigma_ngc[4]} --  W:{sigma_ngc[5]}")

IC 348 summary
means   L:160.44807891635463 --  B:-17.86026392582957 -- D:313.9916389132824
        U:-16.46793416595014 --  V:-6.081614323171151 --  W:-7.585452814726482
        u:-5.3679341659501425 -- v:6.158385676828874 -- w:-0.3354528147264837
sigma   L:0.14983855948177127 --  B:0.22763045736792534 -- D:12.225286987753991
        U:0.7178438176848573 --  V:1.1025115613699512 --  W:1.1634459779129966

NGC 1333 summary
means   L:158.5487420283691 --  B:-20.470342082554822 -- D:293.7665154433183
        U:-16.454469366873358 --  V:-10.090089764675295 --  W:-9.416247915564714
        u:-5.35446936687336 -- v:2.1499102353247306 -- w:-2.166247915564716
sigma   L:0.36148767514621294 --  B:0.2286485679966048 -- D:10.653042013643867
        U:2.609597929009102 --  V:3.27444560812533 --  W:3.837211326620519


In [11]:
c=SkyCoord(l=mu_ic[0]*u.deg,b=mu_ic[1]*u.deg,distance=mu_ic[2]*u.pc, frame = "galactic")

print(c.cartesian.x)
print(c.cartesian.y)
print(c.cartesian.z)

-281.6268882586821 pc
100.01661563185121 pc
-96.30016466616993 pc


In [12]:
c=SkyCoord(l=160.48164348106025*u.deg,b=-17.8049398897683*u.deg,
           distance=320.0214954712528*u.pc, frame = "galactic")

print(c.cartesian.x)
print(c.cartesian.y)
print(c.cartesian.z)

-287.18407489924704 pc
101.80077279513219 pc
-97.85533887445612 pc


In [13]:
print("IC 348 - dispersion")
print(f"UVW: {np.sqrt(sigma_ic[3]**2 + sigma_ic[4]**2 + sigma_ic[5]**2)}")

print("NGC 1333 - dispersion")
print(f"UVW: {np.sqrt(sigma_ngc[3]**2 + sigma_ngc[4]**2 + sigma_ngc[5]**2)}")

IC 348 - dispersion
UVW: 1.7562568243468002
NGC 1333 - dispersion
UVW: 5.679452980420267


## Exporting Results

In [14]:
t = Table()
t["cluster"] = ["IC 348", "NGC 1333"]
t["L"] = [mu_ic[0], mu_ngc[0]]
t["B"] = [mu_ic[1], mu_ngc[1]]
t["D"] = [mu_ic[2], mu_ngc[2]]
t["L_err"] = [sigma_ic[0], sigma_ngc[0]]
t["B_err"] = [sigma_ic[1], sigma_ngc[1]]
t["D_err"] = [sigma_ic[2], sigma_ngc[2]]
t["U"] = [mu_ic[3], mu_ngc[3]]
t["V"] = [mu_ic[4], mu_ngc[4]]
t["W"] = [mu_ic[5], mu_ngc[5]]
t["U_err"] = [sigma_ic[3], sigma_ngc[3]]
t["V_err"] = [sigma_ic[4], sigma_ngc[4]]
t["W_err"] = [sigma_ic[5], sigma_ngc[5]]
t["UVW_dispersion"] = [np.sqrt(sigma_ic[3]**2 + sigma_ic[4]**2 + sigma_ic[5]**2), 
                       np.sqrt(sigma_ngc[3]**2 + sigma_ngc[4]**2 + sigma_ngc[5]**2)]
t["u"] = [mu_ic[3] - osun.U(), mu_ngc[3] - osun.U()]
t["v"] = [mu_ic[4] - osun.V(), mu_ngc[4] - osun.V()]
t["w"] = [mu_ic[5] - osun.W(), mu_ngc[5] - osun.W()]

In [15]:
t

cluster,L,B,D,L_err,B_err,D_err,U,V,W,U_err,V_err,W_err,UVW_dispersion,u,v,w
str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC 348,160.44807891635463,-17.86026392582957,313.9916389132824,0.14983855948177127,0.22763045736792534,12.225286987753991,-16.46793416595014,-6.081614323171151,-7.585452814726482,0.7178438176848573,1.1025115613699512,1.1634459779129966,1.7562568243468002,-5.3679341659501425,6.158385676828874,-0.3354528147264837
NGC 1333,158.5487420283691,-20.470342082554822,293.7665154433183,0.36148767514621294,0.2286485679966048,10.653042013643867,-16.454469366873358,-10.090089764675295,-9.416247915564714,2.609597929009102,3.27444560812533,3.837211326620519,5.679452980420267,-5.35446936687336,2.1499102353247306,-2.166247915564716


In [16]:
t.write("data/deconvolution_results.csv", format = "csv", overwrite=True)